## import packages

In [1]:
import pandas as pd
from tqdm import tqdm
from app.services.evaluator import compute_recall,compute_precision,compute_average_precision,compute_precision_at_k,compute_mrr
from app.services.embeddings_service import EmbeddingSearcher
import os


dataset_name1='nano-beir/arguana'
dataset_name2='beir/webis-touche2020/v2'
dataset_name3='beir/quora/test'
dataset_name4='antique/test'

datasetname=dataset_name4
name=datasetname.replace("/", "-").replace("\\", "_").strip()

/Users/STADIA_AD/Desktop/IR-project/venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Load queries and qrels files

In [2]:
if not datasetname:
    raise ValueError("datasetname variable is not defined")

qrels_df = pd.read_csv(f"data/{name}/qrels.tsv", sep="\t", names=["query_id", "doc_id", "relevance"])
# print(qrels_df)
queries_df = pd.read_csv(f"data/{name}/queries.tsv", sep="\t", names=["query_id", "text"])


# make instance from Embedding

In [3]:
embedding=EmbeddingSearcher(collection_name=name)
print(name)
embedding.load()


MPS backend is available.
antique-test
🔍 Loading embeddings documents


## Evaluation 

In [4]:



all_avg_precisions = []
all_prec_at_10 = []
all_mrr_ranks = []
all_recall = []

for _, row in tqdm(queries_df.iterrows(), total=len(queries_df)):
    query_id = row["query_id"]
    query_text = row["text"]

    # Use your VSM search function
    search_results =embedding.search_vector_index(query_text,top_k=1000)
     # Access the list of result dicts
    result_items = search_results["results"]
    retrieved_docs = [str(doc["doc_id"]) for doc in result_items]

    # Step 2: Convert both retrieved and relevant doc_ids to strings
    relevant_docs = set(qrels_df[qrels_df["query_id"] == query_id]["doc_id"].astype(str))


    # retrieved_docs = [doc["doc_id"] for doc in search_results]

    # Compute metrics
    avg_precision = compute_average_precision(relevant_docs, retrieved_docs)
    prec_at_10 = compute_precision_at_k(relevant_docs, retrieved_docs, k=10)
    recall=compute_recall(relevant_docs, retrieved_docs)

    # Compute rank of first relevant doc for MRR
    rank = 0
    for i, doc_id in enumerate(retrieved_docs):
        if doc_id in relevant_docs:
            rank = i + 1
            break

    all_avg_precisions.append(avg_precision)
    all_prec_at_10.append(prec_at_10)
    all_mrr_ranks.append(rank)
    all_recall.append(recall)

# Final scores
map_score = sum(all_avg_precisions) / len(all_avg_precisions)
mean_prec_at_10 = sum(all_prec_at_10) / len(all_prec_at_10)
mrr_score = compute_mrr(all_mrr_ranks)

print(f"MAP: {map_score:.4f}")
print(f"MRR: {mrr_score:.4f}")
print(f"All Recall values: {all_recall}")
print(f"All Precision@10 values: {all_prec_at_10}")


  0%|          | 0/200 [00:00<?, ?it/s]

100%|██████████| 200/200 [01:07<00:00,  2.98it/s]

MAP: 0.2110
MRR: 0.8435
All Recall values: [0.2777777777777778, 0.7333333333333333, 0.6956521739130435, 0.7804878048780488, 0.7419354838709677, 0.75, 0.45454545454545453, 0.6896551724137931, 0.3548387096774194, 0.2777777777777778, 0.4594594594594595, 0.43478260869565216, 0.88, 0.6451612903225806, 0.6666666666666666, 0.5161290322580645, 0.4782608695652174, 0.6136363636363636, 0.4117647058823529, 0.6333333333333333, 0.325, 0.4666666666666667, 0.5135135135135135, 0.5384615384615384, 0.95, 0.6764705882352942, 0.35294117647058826, 0.4827586206896552, 0.7352941176470589, 0.3, 0.4772727272727273, 0.8888888888888888, 0.84, 0.38461538461538464, 0.9655172413793104, 0.6428571428571429, 0.7346938775510204, 0.7555555555555555, 0.42857142857142855, 0.6206896551724138, 0.5952380952380952, 0.8333333333333334, 0.8260869565217391, 0.6666666666666666, 0.5416666666666666, 0.5882352941176471, 0.23333333333333334, 0.8285714285714286, 0.6923076923076923, 0.4, 0.4, 0.3225806451612903, 0.25, 0.3902439024390244

## save result

In [ ]:
summary = {
    "Mean Average Precision": map_score,
    "Mean Reciprocal Rank": mrr_score
}
import json

output_dir = os.path.join("results", "Embedding", name)
os.makedirs(output_dir)
output_path = os.path.join(output_dir,"evaluation_summary.json")

with open(output_path, "w") as f:
    json.dump(summary, f)

    

In [13]:
# Create DataFrame
df = pd.DataFrame({'All Recall value': all_recall, 'All Precision@10 values': all_prec_at_10})


from pathlib import Path


# Convert string path to Path object
output_dir = Path("results/Embedding/")/name

# Create directory if it doesn't exist
output_dir.mkdir(parents=True, exist_ok=True)

# Save as TSV
df.to_csv(output_dir / 'evaluation.tsv', sep='\t', index=False)

